In [2]:
from transformers import AutoTokenizer, LongT5ForConditionalGeneration

kor2eng_tokenizer = AutoTokenizer.from_pretrained("KETI-AIR-Downstream/long-ke-t5-base-translation-aihub-ko2en")
kor2eng_model = LongT5ForConditionalGeneration.from_pretrained("KETI-AIR-Downstream/long-ke-t5-base-translation-aihub-ko2en")

eng2kor_tokenizer = AutoTokenizer.from_pretrained("KETI-AIR-Downstream/long-ke-t5-base-translation-aihub-en2ko")
eng2kor_model = LongT5ForConditionalGeneration.from_pretrained("KETI-AIR-Downstream/long-ke-t5-base-translation-aihub-en2ko")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.17M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/940 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

https://huggingface.co/docs/transformers/model_doc/t5

In [3]:
# 데이터 불러오기
import pandas as pd

train = pd.read_csv('/data/ephemeral/level2-nlp-datacentric-nlp-06/data/cleaned_train_split.csv')
train.head()

,ID,text,target,url,date
0,ynat-v1_train_04767,해외서 산 요트타고 권총 밀반입 출입국 관리 구멍,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2020.12.07. 오후 12:04
1,ynat-v1_train_01284,신형 유전자 가위 기능 확인 유전병 치료 기대,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.07. 오전 12:01
2,ynat-v1_train_06255,지폐 속 사임당의 그림,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.24. 오후 3:50
3,ynat-v1_train_03516,이란 새 대공방어 시스템 공개 안보 위해 군사력 증강,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.10. 오후 4:30
4,ynat-v1_train_03471,북한 유엔 채택 인권결의에 모든 대응조치 강구 반발,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.11.17. 오후 5:16


In [4]:
back_translate = train.copy()

In [5]:
def translate(text):
    # kor2eng
    input_ids = kor2eng_tokenizer(f"translate_ko2en: {text}", return_tensors="pt").input_ids
    outputs = kor2eng_model.generate(input_ids)
    eng_text = kor2eng_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # eng2kor
    input_ids = eng2kor_tokenizer(f"translate_en2ko: {eng_text}", return_tensors="pt").input_ids
    outputs = eng2kor_model.generate(input_ids)
    backtranslation = eng2kor_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return backtranslation

In [7]:
translate("해외서 산 요트타고 권총 밀반입 출입국 관리 구멍")

/data/ephemeral/.conda/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/data/ephemeral/.conda/lib/python3.10/site-packages/transformers/modeling_utils.py:764: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


'해외에서 태어난 선원, 권총 밀수, 출입국 관리 구멍.'

In [8]:
from tqdm import tqdm
tqdm.pandas()
back_translate['text'] = back_translate['text'].progress_apply(lambda x : translate(x))

  0%|          | 0/4900 [00:00<?, ?it/s]/data/ephemeral/.conda/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/data/ephemeral/.conda/lib/python3.10/site-packages/transformers/modeling_utils.py:764: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
  0%|          | 17/4900 [07:14<39:08:41, 28.86s/it]

In [ ]:
back_translate.to_csv('../data/backtranslation_T5.csv', index=False)